# Proyecto de programación de horarios de cursos universitarios

<b> Diplomatura de Especialización en Desarrollo de Aplicaciones con Inteligencia Artificial</b>

---



**Integrantes:**

* Fiorela Lizárraga
* Frank Ygnacio
* Ricardo Llanos
* Yulian Cama

---

**Objetivos**

-	Minimizar el tiempo de pausa entre clases de los grupos de alumnos
-	Minimizar el tiempo de pausa entre clases de los profesores

**Restricciones**

1. Los recursos no se pueden superponer: ***(Hard constraint)***
    * Ningún profesor puede dar dos clases al mismo tiempo
    * Ningún grupo puede escuchar dos clases al mismo tiempo
    * Ningún salón puede recibir dos clases al mismo tiempo
2. Una clase solo se puede llevar acabo en los salones que tiene permitido ***(Hard constraint)***



##Funciones de Apoyo

### Librerias importadas

In [1]:
import sys
import time
import numpy as np
from random import shuffle, random, sample, randint, randrange, uniform,choice
from copy import deepcopy
import matplotlib.pyplot as plt
import pandas as pd

### Importar archivos de Materias y Docentes


In [2]:
!wget https://raw.githubusercontent.com/LJFiorela/Optimizacion/main/materias_n1.csv
!wget https://raw.githubusercontent.com/LJFiorela/Optimizacion/main/profesores_n1.csv
!wget https://raw.githubusercontent.com/LJFiorela/Optimizacion/main/materias_n2.csv
!wget https://raw.githubusercontent.com/LJFiorela/Optimizacion/main/profesores_n2.csv

--2021-04-03 16:20:16--  https://raw.githubusercontent.com/LJFiorela/Optimizacion/main/materias_n1.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118 [text/plain]
Saving to: ‘materias_n1.csv’

materias_n1.csv     100%[===================>]     118  --.-KB/s    in 0s      

2021-04-03 16:20:16 (3.78 MB/s) - ‘materias_n1.csv’ saved [118/118]

--2021-04-03 16:20:16--  https://raw.githubusercontent.com/LJFiorela/Optimizacion/main/profesores_n1.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 270 [text/plain]
Saving to: ‘profes

### Define el objeto GenClase
Este objeto va a implementar las siguienes funciones:

1.   Initialize : Crea un nuevo cromosoma
2.   mutar_gen : Realiza la mutación en un gen del cromosoma
3.   horas_curso: Devuelve una lista con las horas en las que se dicta el curso. Es una función de apoyo para evaluar cruces de horarios.
4.   evaluacion_cromosoma: Evalua el cromosoma y devuelve diversos valores que se van a usar en el Fitness






In [3]:
#En esta clase se almacena los datos de la clase asi como las funciones para inicializar la clase, hacer mutacion y hallar el fitness
class GenClase(object):
    #se almacenan los dias de la semana y las siglas que se van a usar para armar la cadena del gen  
    dias=['Lunes','Martes','Miercoles','Jueves','Viernes']
    dias_siglas=['LU','MA','MI','JU','VI']
    #horas disponibles en el dia
    horas=[9,10,11,12,13,14,15,16,17,18,19,20,21]
    horas_preferidas=[9,10,11,12,13,14,15]
    #grupos y aulas disponibles
    grupos=['G1','G2','G3','G4','G5']
    aulas=['C1','C2','C3','C4','C5']
    #profesores
    profesores=[]
    profesores_siglas=[]
    profesores_horarios=[]
    #materias y sus siglas correspondientes
    materias_siglas=[]
    materias=[]
    #por cada asignatura se guarda el nombre de la materia, la duracion en horas, en que aulas se pueden dictar y que poesores la pueden dictar
    asignaturas=[]
    #para la funcion initialize se pasa el grupo y el nombre de la asignatura
    
    def importar_csv(self,file_materias,file_profesores):

        df_materias = pd.read_csv(file_materias)
        df_profesores = pd.read_csv(file_profesores)

        df_materias["aulas"]= [cadena.split("|") for cadena in df_materias["aulas"]]
        df_profesores["materias"]= [cadena.split("|") for cadena in df_profesores["materias"]]
        df_profesores["horarios"]= [cadena.split("|") for cadena in df_profesores["horarios"]]
        df_profesores["horarios"] = df_profesores["horarios"].apply(lambda x: [int(y) for y in x])

        conj = []
        for i in df_materias.aulas:
          conj.extend(i)
        lista_aulas = sorted(list(set(conj)))
        self.aulas = lista_aulas
        self.profesores = df_profesores.profesor.values.tolist()
        self.profesores_siglas = df_profesores.codigo.values.tolist()
        self.materias = df_materias.materia.values.tolist()
        self.materias_siglas = df_materias.codigo.values.tolist()

        asignaturas = df_materias.values.tolist()
        prof = df_profesores.values.tolist()

        for i in range(len(asignaturas)):
          mater = asignaturas[i][0]
          p = []
          a = []
          for j in range(len(prof)):
            maters = prof[j][2]
            if mater in maters:
              p.append(prof[j][0])
              a.append(prof[j][1])
          asignaturas[i].append(p)
          asignaturas[i].append(a)
        self.asignaturas = asignaturas

        for p in prof:
          self.profesores_horarios.append([p[1],p[3]])

    def importar_datos(self,file_materias,file_profesores):
        materias_lista = pd.read_csv(file_materias)
        profesores_lista = pd.read_csv(file_profesores)
        # Reemplaza los NaN por 0
        materias_lista = materias_lista.fillna(0)
        profesores_lista = profesores_lista.fillna(0)
        # Convierte las columnas Aula1, Aula2 y Aula 3 en una lista
        aulas = materias_lista[['Aula1','Aula2','Aula3']].values.tolist()
        # Elimina los 0 de las listas
        aulas = [[ele for ele in sub if ele != 0] for sub in aulas]
        # Crea una columna Aula con las listas de aulas
        materias_lista['Aula'] = aulas
        # aulas
        # Combina los datos de profesores con las materias que enseñan en dataframes
        # asignaturasdf = pd.merge(materias, profesores, on='Materia', suffixes=('_materia','_profesor'))
        asignaturasdf1 = pd.merge(materias_lista, profesores_lista, left_on='Materia', right_on=['Materia1'], suffixes=('_materia','_profesor'))
        asignaturasdf2 = pd.merge(materias_lista, profesores_lista, left_on='Materia', right_on=['Materia2'], suffixes=('_materia','_profesor'))
        asignaturasdf3 = pd.merge(materias_lista, profesores_lista, left_on='Materia', right_on=['Materia3'], suffixes=('_materia','_profesor'))
        # Une los dataframes generados anteriormente
        asignaturasdf = pd.concat([asignaturasdf1, asignaturasdf2, asignaturasdf3], ignore_index=True)
        # Elimina columnas innecesarias
        asignaturasdf = asignaturasdf.drop(['Aula1','Aula2','Aula3','Materia1','Materia2','Materia3'], axis=1)
        #Ordena el dataframe por materia
        asignaturasdf.sort_values(by=['Materia'], ignore_index=True)
        # Elimina la repetición materias y crea listas de los datos diferentes
        new = (asignaturasdf.groupby('Materia') # Agrupa por Materia
            .agg({'Codigo_materia':'min','Horas':'min','Aula':'min','Profesor':lambda x: list(x),'Codigo_profesor':lambda x: list(x)}) # Selecciona los datos de las columnas y crea listas
            .reset_index())
        # Convierte el Dataframe en una lista
        self.asignaturas = new.values.tolist()
        self.materias_siglas=[]
        self.materias=[]
        for linea in self.asignaturas:
            self.materias_siglas.append(linea[1])
            self.materias.append(linea[0])
        self.profesores=[]
        self.profesores_siglas=[]
        self.profesores_horarios=[]
        profesores_listado = profesores_lista.values.tolist()
        horarios=[10,11,12,13,14,15]
        for linea2 in profesores_listado:
            profesor_horario=[]
            self.profesores_siglas.append(linea2[1])
            self.profesores.append(linea2[0])
            profesor_horario.append(linea2[1])
            profesor_horario.append(horarios)
            self.profesores_horarios.append(profesor_horario)

    def cambiar_dias(self):

        d = ['Lunes','Martes','Miercoles','Jueves','Viernes','Sabado','Domingo']
        d_siglas = ['LU','MA','MI','JU','VI','SA','DO']
        dia_inicio=''
        dia_fin=''
        while True:
            dia_inicio = input('Ingrese día de inicio: ')
            if dia_inicio not in d:
                print('Ingrese día de inicio válido: Lunes,Martes,Miercoles,Jueves,Viernes,Sabado,Domingo')
            else:
                break
        while True:
            dia_fin = input('Ingrese día de fin: ')
            if dia_fin not in d:
                print('Ingrese día de fin válido: Lunes,Martes,Miercoles,Jueves,Viernes,Sabado,Domingo')
            else:
                break
        while True:
            if (d.index(dia_inicio)==d.index(dia_fin)) or (d.index(dia_inicio)>d.index(dia_fin)):
                print('El día de fin no puede ser igual o menor al día de inicio')
                dia_fin = input('Ingrese día de fin: ')
            else:
                break
        #### Falta la validacion 
        di=d.index(dia_inicio)
        df=d.index(dia_fin)+1
        # Seleccionamos los días y sus siglas
        dias = d[di:df]
        dias_siglas = d_siglas[di:df]
        self.dias=dias
        self.dias_siglas=dias_siglas
    
    def cambiar_horario(self):
        while True:
            try:
                hi = int(input('Ingrese hora de inicio: '))
            except ValueError:
                print("Debes escribir un número.")
                continue

            if hi < 0 or hi > 24:
                print("Debes escribir una hora entre 0 y 24.")
                continue
            else:
                break

        while True:
            try:
                hf = int(input('Ingrese hora de fin: '))
            except ValueError:
                print("Debes escribir un número.")
                continue

            if hf < hi:
                print("Debes ser mayor a la hora inicial.")
                continue
            if hf == hi:
                print("No puede ser la misma hora inicial.")
                continue
            else:
                break
        horas = [i for i in range(hi,hf+1)]
        self.horas=horas

    def cambiar_horas_premium(self):
        while True:
            try:
                hpi = int(input('Ingrese hora preferida de inicio: '))
            except ValueError:
                print("Debes escribir un número.")
                continue

            if hpi < min(self.horas) or hpi > max(self.horas):
                print("Debes escribir una hora entre el rango inicial.")
                continue
            else:
                break

        while True:
            try:
                hpf = int(input('Ingrese hora preferida de fin: '))
            except ValueError:
                print("Debes escribir un número.")
                continue
            if hpf <  min(self.horas) or hpf > max(self.horas):
                print("Debes escribir una hora entre el rango inicial.")
                continue
            if hpf == hpi:
                print("No puede ser la misma hora preferida inicial.")
                continue
            else:
                break
        horas_premium = [i for i in range(hpi,hpf+1)]
        self.horas_preferidas=horas_premium

    def cambiar_grupos(self):
        while True:
            try:
                cant_grupos = int(input('Ingrese cantidad de grupos: '))
            except ValueError:
                print("Debes escribir un número.")
                continue
            break
        grupos = []
        i=1
        while i <= cant_grupos:
            grupos.append('G'+ str(i))
            i+=1
        self.grupos=grupos

    def initialize(self,grupo1,asignatura1):
        self.grupo = grupo1
        self.asignatura = asignatura1
        #se selecciona el dia de la semana de forma aleatoria
        self.dia=self.dias[randrange(0, len(self.dias))]
        #se recupear el indice de la asignatura y con este indice hallamos la duracion de dicha asignatura
        indice_asignatura=self.materias_siglas.index(asignatura1)
        duracion=self.asignaturas[indice_asignatura][2]
        #se selecciona una hora de forma aleatoria de tal forma que considerando su duracion el curso se dicte en el mismo dia
        #por ejemplo si el curso dura 3 horas la fecha de inicia no puede ser las 9 de la noche ya que se pasaría del horario laboral 
        self.hora=self.horas[randrange(0, len(self.horas)-(duracion-1))]
        #selecciona de forma aleatoria uno de los profesores disponibles para dicha asignatura
        self.profesor=self.asignaturas[indice_asignatura][5][randrange(0, len(self.asignaturas[indice_asignatura][5]))]
        #selecciona de forma aleatoria entre la areas disponibles para la asignatura
        self.aula=self.asignaturas[indice_asignatura][3][randrange(0, len(self.asignaturas[indice_asignatura][3]))]
        #transforma la hora a string
        if self.hora<10:
          hora_str='0'+str(self.hora)
        else:
          hora_str=str(self.hora)
        #crea una cadena resumida que seria : Grupo + Asignatura + Día de la semana+ Hora + Profesor + Aula
        cadena_gen=self.grupo[0:2]+self.asignatura[0:2].upper()+self.dia[0:2].upper()+hora_str+self.profesor.upper()+self.aula[0:2].upper()
        return cadena_gen
    def mutar_gen(self,alelo):
        #decide que tipo de mutacion: se cambiara el dia, la hora, el profesor el aula (no se cambia el grupo y la materia ya que es una permutacion)
        tipo_mutacion=choice(['dia','hora','profesor','aula'])
        #escoge un dia de forma aleatoria
        if tipo_mutacion == 'dia':
            alelo_mutado=alelo[0:4]+choice(self.dias_siglas).upper()+alelo[6:len(alelo)]
        #recupera la duracion del curso del alelo y calcula una hora de forma aleatoria de tal forma el curso se dicte entre las horas disponibles
        #(que no se salga del horario)
        if tipo_mutacion == 'hora':
            indice_asignatura=self.materias_siglas.index(alelo[2:4])
            duracion=self.asignaturas[indice_asignatura][2]
            hora_sel=self.horas[randrange(0, len(self.horas)-(duracion-1))]
            if hora_sel<10:
                hora_sel_str='0'+str(hora_sel)
            else: 
                hora_sel_str=str(hora_sel)
            alelo_mutado=alelo[0:6]+hora_sel_str.upper()+alelo[8:len(alelo)]
        #extrae de forma aleatoria un profesor disponible para la asignatura, para armar la cadena se extrae los dos primeros caracteres del nombe del profesor
        if tipo_mutacion == 'profesor':
            indice_asignatura=self.materias_siglas.index(alelo[2:4])
            alelo_mutado=alelo[0:8]+choice(self.asignaturas[indice_asignatura][5])[0:2].upper()+alelo[10:len(alelo)]
        #extrae un aula disponible para ese curso de forma aleatoria
        if tipo_mutacion == 'aula':
            indice_asignatura=self.materias_siglas.index(alelo[2:4])
            alelo_mutado=alelo[0:10]+choice(self.asignaturas[indice_asignatura][3])[0:2].upper()
        return alelo_mutado  
    
    #recupera un conjunto con las horas en las que se dicta el curso, por ejemplo si el curso inicia a las 10 y dura 
    #3 horas el conjunto seria (10,11,12)
    def horas_curso(self,hora_inicio,curso):
        hora_inicio_int=int(hora_inicio)
        indice_asignatura=self.materias_siglas.index(curso)
        duracion_curso=self.asignaturas[indice_asignatura][2]
        horas_ocupadas=[]
        for x in range(hora_inicio_int,hora_inicio_int+duracion_curso):
              horas_ocupadas.append(x)
        return set(horas_ocupadas)

    def evaluacion_cromosoma(self,cromosoma):
        #inicializa los conflictos en cero
        conflictos = 0
        horas_premium = 0
        profesor_extras = 0
        profesor_horas={}
        grupo_horas={}
        #genera 2 diccionarios, el profesor_horas, se almacenara cada prrofesor con sus respectivos dias y horas en los que tienen cursos asignados
        #en grupo_horas se realiza lo mismo por cada grupo
        dias_no_usados=deepcopy(self.dias_siglas)

        for i in range(0, len(cromosoma)):
            #por cada gen se analiza el dia y la hora y con ese dato vamos llenando los 2 diccionarios 
            #para esto usamos la funcion horas_curso para considerar las horas adicionales si el curso tiene una duracion mayor a una hora
            cGrupo=cromosoma[i][0:2] 
            cCurso=cromosoma[i][2:4]
            cDia=cromosoma[i][4:6]  
            if cDia in dias_no_usados: 
               dias_no_usados.remove(cDia)
            cHora=cromosoma[i][6:8]
            cProfesor=cromosoma[i][8:10] 
            cClase=cromosoma[i][10:12] 
            if cProfesor not in profesor_horas:
                 profesor_horas[cProfesor]={}
            if cDia not in profesor_horas[cProfesor]:
                 profesor_horas[cProfesor][cDia]=[]
                 profesor_horas[cProfesor][cDia].extend(self.horas_curso(cHora,cCurso))
            else:
                 profesor_horas[cProfesor][cDia].extend(self.horas_curso(cHora,cCurso))
            if cGrupo not in grupo_horas:
                 grupo_horas[cGrupo]={}
            if cDia not in grupo_horas[cGrupo]:
                 grupo_horas[cGrupo][cDia]=[]
                 grupo_horas[cGrupo][cDia].extend(self.horas_curso(cHora,cCurso))
            else:
                 grupo_horas[cGrupo][cDia].extend(self.horas_curso(cHora,cCurso))

            horas_bloque=self.horas_curso(cHora,cCurso)
            # print('-----')
            # print(horas_bloque)
            # print(cProfesor)    
            # print(self.profesores_horarios[self.profesores_siglas.index(cProfesor)][1])

            horas_premium = horas_premium + len(horas_bloque&set(self.horas_preferidas))
            cruce_horarios=len(horas_bloque&set(self.profesores_horarios[self.profesores_siglas.index(cProfesor)][1]))
            profesor_extras+=len(horas_bloque)-cruce_horarios

            # print(len(horas_bloque)-cruce_horarios)

            for j in range(0, len(cromosoma)) :
                if (j>=i):
                  #para comparar se valida que no sea el mismo gen
                  if cromosoma[i][0:4] != cromosoma[j][0:4]:
                    #a continuacion se valida si es que el dia y las horas se cruzan entre ambos genes
                    if (cDia == cromosoma [j][4:6]) and (horas_bloque&self.horas_curso(cromosoma[j][6:8],cromosoma[j][2:4]))!=set() :
                        #si es así se valida que no sean del mismo grupo, que se dicten en la misma clase o tengan el mismo profesor
                        #si los 3 valores son diferentes no deberia haber cruce ni generarse ningun conflicto
                        if (cGrupo == cromosoma[j][0:2]): conflictos += 1
                        if (cProfesor == cromosoma[j][8:10]): conflictos += 1
                        if (cClase == cromosoma[j][10:12]): conflictos += 1
                        

        #se revisa el diccionario por cada profesor
        #por cada dia que dicta el profesor se revise la primera y ultima hora de dictado 
        # y se valida en el diccionario, si hay horas que no estan asignadas se suma a las horas libres
        horas_libres_profesores=0
        for profe in profesor_horas:
           dias_ocupados_profesores=0 
           for dia in profesor_horas[profe]:
              dias_ocupados_profesores=dias_ocupados_profesores+1
              hora_min=min(profesor_horas[profe][dia])
              hora_max=max(profesor_horas[profe][dia])
              for x in range(hora_min,hora_max+1):
                if x not in profesor_horas[profe][dia]:
                  horas_libres_profesores=horas_libres_profesores+1
        horas_libres_grupos=0
        #lo mimsmo se hace en el diccionario de grupo
        for grup in grupo_horas:
           dias_ocupados_grupos=0  
           for dia in grupo_horas[grup]:
              dias_ocupados_grupos=dias_ocupados_grupos+1
              hora_min=min(grupo_horas[grup][dia])
              hora_max=max(grupo_horas[grup][dia])
              for x in range(hora_min,hora_max+1):
                if x not in grupo_horas[grup][dia]:
                  horas_libres_grupos=horas_libres_grupos+1
        #devuelve el numero de conflictos,y horas libres
        d_no_usados=len(dias_no_usados)          
        return conflictos, horas_libres_profesores, horas_libres_grupos, horas_premium, dias_ocupados_profesores, dias_ocupados_grupos, d_no_usados,profesor_extras

### Carga de datos y Pruebas


In [4]:
Genes=GenClase()
Genes.importar_csv('materias_n1.csv','profesores_n1.csv')
Genes.cambiar_dias()
Genes.cambiar_horario()
Genes.cambiar_horas_premium()
Genes.cambiar_grupos()

print(Genes.profesores)
print(Genes.profesores_siglas) 

Ingrese día de inicio: Lunes
Ingrese día de fin: Martes
Ingrese hora de inicio: 8
Ingrese hora de fin: 22
Ingrese hora preferida de inicio: 8
Ingrese hora preferida de fin: 16
Ingrese cantidad de grupos: 5
['Juan Perez', 'Simon Smith', 'Joe Pelaez', 'Pedro Bracamonte', 'Zacarias Espejo']
['JP', 'SS', 'JZ', 'PB', 'ZE']


In [5]:
gen=Genes.initialize('G1','BI')
gen2=Genes.mutar_gen(gen)
print(gen)
print(gen2)
horas=Genes.horas_curso('09','BI')
print(horas)
#cromosoma=['G1MAVI09PEC1','G1LEVI13PEC1','G3BIVI18JOC1','G1MALU09JOC1','G1LEVI11JOC1','G3BIVI13PEC1']
#print(Genes.evaluacion_cromosoma(cromosoma))

G1BIMA10PBC4
G1BILU10PBC4
{9, 10}


###Funcion para imprimir el horario resultante


In [6]:
def imprimir_horario(solucion):
    
    horario=[]
    #c=GenClase()
    #genera una lista de listas de aulas x dia y hora
    fila=[]
    fila.append('Aulas')
    for d in Genes.aulas:
        fila.append(d+'         ')
    horario.append(fila)  
    cantidad_horas=len(Genes.horas)*len(Genes.dias)
    for hora_dia in range(0,cantidad_horas):
        fila=[]
        hora_actual=Genes.horas[hora_dia % len(Genes.horas)]
        fila.append(hora_actual)
        for salon in Genes.aulas:
            fila.append('____________')
        horario.append(fila)

    #llena la matriz con las clases en su hora respectiva considerando la duracion
    #si la clase dura 3 horas se repite 3 veces esa clase considerando hora de inicio y fin
    for gen in solucion:
        x=0
        y=0
        y=Genes.aulas.index(gen[10:12])+1
        dia=Genes.dias_siglas.index(gen[4:6])
        hora=Genes.horas.index(int(gen[6:8]))
        indice_asignatura=Genes.materias_siglas.index(gen[2:4])
        duracion=Genes.asignaturas[indice_asignatura][2]
        indice_hora=((len(Genes.horas)*dia)+hora)+1
        for x in range(indice_hora,duracion+indice_hora):
            horario[x][y]=gen

    #imprime el horario
    f=0
    d1=0
    saltos_linea=[]
    s=0
    for salto in Genes.dias:
        saltos_linea.append(s*len(Genes.horas))
        s=s+1
    for fila in horario:
        print(fila)
        if f in saltos_linea:
            print('--------------------------------------------------------------------------------------')
            print('                      '+Genes.dias[d1])
            print('--------------------------------------------------------------------------------------')
            d1=d1+1
        f=f+1

### Funcion para Exportar Horario


In [8]:
def exportar_horario(solucion):
    
    horario=[]
    #c=GenClase()
    #genera una lista de listas de aulas x dia y hora
    fila=[]
    clss=[]
    fila.append('Horario')
    for d in Genes.aulas:
        clss.append(d)
    horario.append(fila)  
    cantidad_horas=len(Genes.horas)*len(Genes.dias)
    for hora_dia in range(0,cantidad_horas):
        fila=[]
        hora_actual=Genes.horas[hora_dia % len(Genes.horas)]
        fila.append(hora_actual)
        for salon in Genes.aulas:
            fila.append(' ')
        horario.append(fila)

    #llena la matriz con las clases en su hora respectiva considerando la duracion
    #si la clase dura 3 horas se repite 3 veces esa clase considerando hora de inicio y fin
    for gen in solucion:
        x=0
        y=0
        y=Genes.aulas.index(gen[10:12])+1
        dia=Genes.dias_siglas.index(gen[4:6])
        hora=Genes.horas.index(int(gen[6:8]))
        indice_asignatura=Genes.materias_siglas.index(gen[2:4])
        duracion=Genes.asignaturas[indice_asignatura][2]
        indice_hora=((len(Genes.horas)*dia)+hora)+1
        for x in range(indice_hora,duracion+indice_hora):
            # if gen[0:2] == Genes.profesores_siglas.index(gen[0:2])
            nom = Genes.profesores[Genes.profesores_siglas.index(gen[8:10])]
            mat = Genes.materias[Genes.materias_siglas.index(gen[2:4])]
            grp = gen[0:2]
            # Solo codigos
            # horario[x][y]=gen[0:4]+gen[8:10] 
            # Detallado
            horario[x][y]= grp+'-'+mat+'-'+nom
    
    
    # Añade dias
    f=0
    d1=0
    saltos_linea=[]
    s=0
    for salto in Genes.dias:
        saltos_linea.append(s*len(Genes.horas))
        s=s+1
    for fila in horario:
        if f in saltos_linea:
            horario.insert(f+1+d1,[Genes.dias[d1]]+clss)
            d1=d1+1
        f=f+1

    return pd.DataFrame(horario)

##Funciones Comunes Algoritmo Genético

### Define la estructura de un individuo en el AG con sus operadores genéticos 

Implementa el individuo del AG. Un individuo tiene un cromosoma es una lista de G X M genes. El individuo se crea con un for anidado de grupos y materias, de tal forma que no se repita una materia para un mismo grrupo.

*   G=número de grupos
*   M=número de materias

Como el orden de los genes es importante se esta considerando los algoritmos de cruce crossover_onepoint
y crossover_uniform que no alteran el orden

In [9]:
class Individual:

    def __init__(self, chromosome):  # el constructor recibe un cromosoma
        self.chromosome = chromosome[:]  
        self.fitness = -100  # -1 indica que el individuo no ha sido evaluado

    #no se cambio crossover_onepoint ni crossover_uniform

    def crossover_onepoint(self, other):
        "Retorna dos nuevos individuos del cruzamiento de un punto entre individuos self y other "
        c = randrange(len(self.chromosome))
        ind1 = Individual(self.chromosome[:c] + other.chromosome[c:])
        ind2 = Individual(other.chromosome[:c] + self.chromosome[c:])
        return [ind1, ind2]   
    
    def crossover_uniform(self, other):
        chromosome1 = []
        chromosome2 = []
        "Retorna dos nuevos individuos del cruzamiento uniforme entre self y other "
        for i in range(len(self.chromosome)):
            if uniform(0, 1) < 0.5:
                chromosome1.append(self.chromosome[i])
                chromosome2.append(other.chromosome[i])
            else:
                chromosome1.append(other.chromosome[i])
                chromosome2.append(self.chromosome[i])
        ind1 = Individual(chromosome1)
        ind2 = Individual(chromosome2)
        return [ind1, ind2] 

 
    #se cambio este codigo 
    def mutation_flip(self):
        "Cambia aleatoriamente el alelo de un gen."
        #gen=GenClase()
        new_chromosome = deepcopy(self.chromosome)
        mutGene = randrange(0,len(new_chromosome))   # escoge un gen para mutar
        #envia el gen a la funcion mutar_gen y se recibe el gen mutado
        new_chromosome[mutGene]=Genes.mutar_gen(new_chromosome[mutGene])
        return Individual(new_chromosome)

    def mutation_multiflip(self):
        """
        Cambia los alelos de un conjunto de genes escogidos aleatoriamente (hasta un maximo de 50% de genes).
        """
        #gen=GenClase()
        new_chromosome = deepcopy(self.chromosome)
        #se genera una cantidad de genes a mutar de forma aleatoria que no pase del 50%
        cantidadGene= randrange(0,(len(new_chromosome)//2))   
        #lista de indices de genes mutados
        genesmutados=[]
        for x in range(cantidadGene):
            # escoge un gen para mutar de forma aleatoria
            mutGene = randrange(0,len(new_chromosome))   
            #se verifica que ese gen no halla sido mutado anteriormente, y en caso contrario se realiza la mutacion
            while mutGene in genesmutados:
                mutGene = randrange(0,len(new_chromosome)) 
                new_chromosome[mutGene]=Genes.mutar_gen(new_chromosome[mutGene])
            genesmutados.append(mutGene)
        return Individual(new_chromosome)

       

### Funcion para obtener los fitnesses de un cromosoma (Multiobjetivo)


In [10]:
#FUNCION FITNESS MULTIOBJETIVO
def get_fitness_multi(chromosome):
     fitness = np.zeros(2)
     #c=GenClase()
     conflictos, horas_libres_profesores, horas_libres_grupo , horas_premium, dias_ocupados_profesores, dias_ocupados_grupos,dias_no_usados, profesor_extras  =Genes.evaluacion_cromosoma(chromosome)
     #castiga el doble al fitness del individuo que tiene algun confflictor para que estos individuos sean descartados rapidamente
     fitness[0] = (dias_ocupados_profesores)+(dias_ocupados_grupos)+ (horas_premium) - (dias_no_usados) - (profesor_extras) - (horas_libres_profesores*30)-(100*conflictos)
     fitness[1] = (dias_ocupados_profesores)+(dias_ocupados_grupos)+ (horas_premium) - (dias_no_usados) - (profesor_extras) - (horas_libres_grupo*30)-(100*conflictos)
     return fitness


### Funcion para obtener los fitnesses de un cromosoma (Mono-objetivo)

**Se implemento el fitness_mono_objetivo para usarlo con al algoritmo genetico mono objetivo y devuelve un solo valor**

In [11]:
#FUNCION FITNESS MONO-OBJETIVO
def get_fitness_mono(chromosome):
     fitness = 0 
     #c=GenClase()
     conflictos, horas_libres_profesores, horas_libres_grupo, horas_premium, dias_ocupados_profesores, dias_ocupados_grupos,dias_no_usados, profesor_extras   =Genes.evaluacion_cromosoma(chromosome)
     #castiga el doble al fitness del individuo que tiene algun confflictor para que estos individuos sean descartados rapidamente
     #se considera la suma de horas libres de profesores y grupos como un solo objetivo
     fitness = (dias_ocupados_profesores)+(dias_ocupados_grupos)+ (horas_premium) - (dias_no_usados) - (profesor_extras) - (horas_libres_profesores*30)- (horas_libres_grupo*30)-(100*conflictos)
     return fitness

### Funcion para inicializar aleatoriamente una población de individuos

In [12]:
def init_population(pop_size):
    #Inicializa una poblacion de pop_size individuos, cada cromosoma de individuo de tamaño chromosome_size.
    #c=GenClase()
    population = []
    for i in range(pop_size):
      new_chromosome=[]
      #genera genes haciendo una permutaciuon de grupos y materias
      for grupo in Genes.grupos:
        for materias in Genes.materias_siglas:
            #por cada gen escoge de forma alatoria el profesor, dia, hora y clase
            new_chromosome.append(Genes.initialize(grupo,materias))
      population.append(Individual(new_chromosome))  
    return population


### Funcion para evaluar una población de individuos (Multi Objetivo)

In [13]:
def evaluate_population_multi(population):
    """ Evalua una poblacion de individuos con la funcion get_fitness """
    pop_size = len(population)

    for i in range(pop_size):
        if population[i].fitness == -100:    # evalua solo si el individuo no esta evaluado
            population[i].fitness = get_fitness_multi(population[i].chromosome)
            

### Funcion para evaluar una población de individuos (Mono Objetivo)

esta función evaluate llama la funcion fitness mono objetivo

In [14]:
def evaluate_population_mono(population):
    """ Evalua una poblacion de individuos con la funcion get_fitness """
    popsize = len(population)
    for i in range(popsize):
        if population[i].fitness == -100:    # evalua solo si el individuo no esta evaluado
            population[i].fitness = get_fitness_mono(population[i].chromosome)

##Funciones Algoritmo Genético - Multiobjetivo

### Funcion para crear la población hija (multiobjetivo)

In [15]:
def build_offspring_population(population, crossover, mutation, pmut):     
    """ Construye una poblacion hija con los operadores de cruzamiento y mutacion pasados
        crossover:  operador de cruzamiento
        mutation:   operador de mutacion
        pmut:       taza de mutacion
    """
    pop_size = len(population)
    
    ## Selecciona parejas de individuos (mating_pool) para cruzamiento 
    
    mating_pool = []
    for i in range(int(pop_size/2)): 
        # escoje dos individuos diferentes aleatoriamente de la poblacion
        permut = np.random.permutation( pop_size )
        mating_pool.append( (population[permut[0]], population[permut[1]] ) ) 
        
    ## Crea la poblacion descendencia cruzando las parejas del mating pool 
    offspring_population = []
    for i in range(len(mating_pool)): 
        if crossover == "onepoint":
            offspring_population.extend( mating_pool[i][0].crossover_onepoint(mating_pool[i][1]) ) # cruzamiento 1 punto
        elif crossover == "uniform":
            offspring_population.extend( mating_pool[i][0].crossover_uniform(mating_pool[i][1]) ) # cruzamiento uniforme
        else:
            raise NotImplementedError

    ## Aplica el operador de mutacion con probabilidad pmut en cada hijo generado
    for i in range(len(offspring_population)):
        if uniform(0, 1) < pmut: 
            if mutation == "flip":
                offspring_population[i] = offspring_population[i].mutation_flip() # cambia el alelo de un gen
            elif mutation == "multiflip":
                offspring_population[i] = offspring_population[i].mutation_multiflip() # cambia el alelo de varios genes
            else:
                raise NotImplementedError   
                
    return offspring_population

### Funcion para hallar la distancia Crowding (Multiobjetivo)


In [16]:
def get_crowding_distances(fitnesses):
    """
    La distancia crowding de un individuo es la diferencia del fitness mas proximo hacia arriba menos el fitness mas proximo 
    hacia abajo. El valor crowding total es la suma de todas las distancias crowdings para todos los fitness
    """
    
    pop_size = len(fitnesses[:, 0])
    num_objectives = len(fitnesses[0, :])

    # crea matriz crowding. Filas representan individuos, columnas representan objectives
    crowding_matrix = np.zeros((pop_size, num_objectives))

    # normalisa los fitnesses entre 0 y 1 (ptp es max - min)
    normalized_fitnesses = (fitnesses - fitnesses.min(0)) / fitnesses.ptp(0)

    for col in range(num_objectives):   # Por cada objective
        crowding = np.zeros(pop_size)

        # puntos extremos tienen maximo crowding
        crowding[0] = 1
        crowding[pop_size - 1] = 1

        # ordena los fitness normalizados del objectivo actual
        sorted_fitnesses = np.sort(normalized_fitnesses[:, col])
        sorted_fitnesses_index = np.argsort(normalized_fitnesses[:, col])

        # Calcula la distancia crowding de cada individuo como la diferencia de score de los vecinos
        crowding[1:pop_size - 1] = (sorted_fitnesses[2:pop_size] - sorted_fitnesses[0:pop_size - 2])

        # obtiene el ordenamiento original
        re_sort_order = np.argsort(sorted_fitnesses_index)
        sorted_crowding = crowding[re_sort_order]

        # Salva las distancias crowdingpara el objetivo que se esta iterando
        crowding_matrix[:, col] = sorted_crowding

    # Obtiene las distancias crowding finales sumando las distancias crowding de cada objetivo 
    crowding_distances = np.sum(crowding_matrix, axis=1)

    return crowding_distances

### Funcion selección según la distancia Crowding (Multiobjetivo)


In [17]:
def select_by_crowding(population, num_individuals):
    """
    Selecciona una poblacion de individuos basado en torneos de pares de individuos: dos individuos se escoge al azar
    y se selecciona el mejor segun la distancia crowding. Se repite hasta obtener num_individuals individuos
    """    
    population = deepcopy(population)
    pop_size = len(population)
    
    num_objectives = len(population[0].fitness)
    
    # extrae los fitness de la poblacion en la matriz fitnesses
    fitnesses = np.zeros([pop_size, num_objectives])
    for i in range(pop_size): fitnesses[i,:] = population[i].fitness
        
    # obtiene las  distancias  crowding
    crowding_distances = get_crowding_distances(fitnesses)   
    
    population_selected = []   # poblacion escogida

    for i in range(num_individuals):  # por cada individuo a seleccionar

        # escoje dos individuos aleatoriamente de la poblacion no escogida aun
        permut = np.random.permutation( len(population) )
        ind1_id = permut[0]
        ind2_id = permut[1]

        # Si ind1_id es el mejor
        if crowding_distances[ind1_id] >= crowding_distances[ind2_id]:

            # traslada el individuo ind1 de population a la lista de individuos seleccionados
            population_selected.append( population.pop(ind1_id) )
            # remueve la distancia crowding del individuo seleccionado
            crowding_distances = np.delete(crowding_distances, ind1_id, axis=0)
            
        else:  # Si ind2_id es el mejor
            
            # traslada el individuo ind2 de population a la lista de individuos seleccionados
            population_selected.append( population.pop(ind2_id) )
            # remueve la distancia crowding del individuo seleccionado
            crowding_distances = np.delete(crowding_distances, ind2_id, axis=0)

    return (population_selected)

### Funcion obtiene población de individuos de la frontera de Pareto (Multiobjetivo)

In [18]:
def get_paretofront_population(population):
    """
    Obtiene de population la poblacion de individups de la frontera de Pareto, 
    """
    population = deepcopy(population)
    pop_size = len(population)
    
    # todos los individuos son inicialmente asumidos como la frontera de Pareto
    pareto_front = np.ones(pop_size, dtype=bool)
    
    for i in range(pop_size): # Compara cada individuo contra todos los demas
        for j in range(pop_size):
            # Chequea si individuo 'i' es dominado por individuo 'j'
            #if all(population[j].fitness >= population[i].fitness) and any(population[j].fitness > population[i].fitness):
            #if str(all(population[j].fitness >= population[i].fitness)) and str(any(population[j].fitness > population[i].fitness)):
            if all(np.asarray(population[j].fitness) >= np.asarray(population[i].fitness)) and any(np.asarray(population[j].fitness) > np.asarray(population[i].fitness)):
                # j domina i -> señaliza que individuo 'i' como no siendo parte de la frontera de Pareto
                pareto_front[i] = 0
                break   # Para la busqueda para 'i' (no es necesario hacer mas comparaciones)

    paretofront_population = []
    for i in range(pop_size):  # construye la lista de individuos de la frontera de Pareto 
        if pareto_front[i] == 1: paretofront_population.append(population[i])
        
    return paretofront_population

### Funcion para construir la nueva generación (Multiobjetivo)


In [19]:
def build_next_population(population, min_pop_size, max_pop_size):
    """
    Construye la poblacion de la siguiente generacion añadiendo sucesivas fronteras de Pareto hasta 
    tener una poblacion de al menos min_pop_size individuos. Reduce la frontera de Pareto con el metodo de
    crowding distance si al agregar la frontera excede el tamaño maximo de la poblacion (max_pop_size)
    """
    population = deepcopy(population)
    pareto_front = []
    next_population = []
    
    while len(next_population) < min_pop_size:   # mientras la poblacion no tenga el tamaño minimo
        # obtiene la poblacion frontera de Pareto actual
        paretofront_population = get_paretofront_population(population)
        
        # si poblacion actual + paretofront excede el maximo permitido -> reduce paretofront con el metodo de crowding
        combined_population_size = len(next_population) + len(paretofront_population)
        if  combined_population_size > max_pop_size:
            paretofront_population = select_by_crowding( paretofront_population, max_pop_size-len(next_population) ) 
        
        # Adiciona la frontera de Pareto (original o reducida) a la poblacion en construccion
        next_population.extend( paretofront_population )
    
        # remueve de population los individuos que fueron agregados a next_population 
        for i in range( len(paretofront_population) ):
            for j in range( len(population) ):
                if all( np.asarray(paretofront_population[i].chromosome) == np.asarray(population[j].chromosome) ):
                    del(population[j])
                    break
                    
    return next_population

### Algoritmo genetico multiobjetivo (NSGA-II) principal

In [20]:
## CODIGO PRINCIPAL DEL  ALGORITMO GENETICO  NSGA-II
def genetic_algorithm_multi(population, ngen=100, pmut=0.1, 
                      crossover="uniform", mutation="flip", 
                      min_pop_size=100, 
                      max_pop_size=100):

    ## Ejecuta los ciclos evolutivos 
    for g in range(ngen):   # Por cada generacion
    
        if g %10 == 0:
            print ('Generacion {} (de {}) '.format(g, ngen))
    
        ## genera y evalua la poblacion hija    
        Q = build_offspring_population(population, crossover, mutation, pmut)
        evaluate_population_multi(Q)

        ## une la poblacion padre y la poblacion hija
        population.extend(Q) 

        ## Construye la poblacion de la siguiente generacion
        population = build_next_population(population, MIN_POP_SIZE, MAX_POP_SIZE)

    # Obtiene la poblacion de la frontera de pareto final 
    pareto_front_population = get_paretofront_population(population)
    return pareto_front_population

#pareto_front_population=genetic_algorithm_multi(P, GENERATIONS, PMUT,"uniform", "flip",MIN_POP_SIZE, MAX_POP_SIZE)


##Funciones Algoritmo Genético - Mono-Objetivo

### Función - Selección de padres por Ruleta

In [21]:
def select_parents_roulette(population):
    popsize = len(population)
    
    # Escoje el primer padre
    sumfitness = sum([indiv.fitness for indiv in population])  # suma total del fitness de la poblacion
    pickfitness = uniform(0, sumfitness)   # escoge un numero aleatorio entre 0 y sumfitness
    cumfitness = 0     # fitness acumulado
    for i in range(popsize):
        cumfitness += population[i].fitness
        if cumfitness > pickfitness: 
            iParent1 = i
            break
    
    # Escoje el segundo padre, desconsiderando el primer padre
    sumfitness = sumfitness - population[iParent1].fitness # retira el fitness del padre ya escogido
    pickfitness = uniform(0, sumfitness)   # escoge un numero aleatorio entre 0 y sumfitness
    cumfitness = 0     # fitness acumulado
    for i in range(popsize):
        if i == iParent1: continue   # si es el primer padre 
        cumfitness += population[i].fitness
        if cumfitness > pickfitness: 
            iParent2 = i
            break        
    return (population[iParent1], population[iParent2])

### Función - Selección de padres por Torneo

In [22]:
def select_parents_tournament(population, tournament_size):
    # Escoje el primer padre
    list_indiv=[]
    x1 = np.random.permutation(len(population) )
    y1= x1[0:tournament_size]
    for i in range(tournament_size):
        list_indiv.append(population[y1[i]].fitness)
    
    iParent1=np.argmax(list_indiv)
    
    # Escoje el segundo padre, desconsiderando el primer padre   
    x2 = np.delete(x1, iParent1)
    x2 = np.random.permutation(x2)
    list_indiv=[]
    y2= x2[0:tournament_size]
    for i in range(tournament_size):
        list_indiv.append(population[y2[i]].fitness)
    iParent2=np.argmax(list_indiv)
    
    return (population[x1[iParent1]],population[x2[iParent2]])

### Función - Selección de sobrevivientes por Ranking

In [23]:
def select_survivors_ranking(population, offspring_population, numsurvivors):
    next_population = []
    population.extend(offspring_population) # une las dos poblaciones
    isurvivors = sorted(range(len(population)), key=lambda i: population[i].fitness, reverse=True)[:numsurvivors]
    for i in range(numsurvivors):
        next_population.append(population[isurvivors[i]])
    return next_population

### Algoritmo genetico Mono Objetivo principal

In [24]:
def genetic_algorithm_mono(population, ngen=100, pmut=0.1, 
                      crossover="onepoint", mutation="flip", 
                      selection_parents_method="roulette", 
                      selection_survivors_method="ranking"):
    """Algoritmo Genetico para el problema de la mochila
        ngen:       maximo numero de generaciones 
        pmut:       tasa de mutacion
        crossover:  operador de cruzamiento
        mutation:   operador de mutacion
        selection_parents_method: método de selección de padres para cruzamiento
        selection_survivors_method: método de selección de sobrevivientes 
    """
    
    popsize = len(population)
    evaluate_population_mono(population)  # evalua la poblacion inicial
    ibest = sorted(range(len(population)), key=lambda i: population[i].fitness, reverse=True)[:1]  # mejor individuo
    bestfitness = [population[ibest[0]].fitness]  # fitness del mejor individuo
    print("Poblacion inicial, best_fitness = {}".format(population[ibest[0]].fitness))
    
    for g in range(ngen):   # Por cada generacion

        ## Selecciona parejas de individuos (mating_pool) para cruzamiento con el metodo de la ruleta
        mating_pool = []
        for i in range(int(popsize/2)):
            if selection_parents_method == "roulette":
                mating_pool.append(select_parents_roulette(population))
            elif selection_parents_method == "tournament":
                mating_pool.append(select_parents_tournament(population, 3))
            else:
                raise NotImplementedError
        ## Crea la poblacion descendencia cruzando las parejas del mating pool 
        offspring_population = []
        for i in range(len(mating_pool)): 
            if crossover == "onepoint":
                offspring_population.extend( mating_pool[i][0].crossover_onepoint(mating_pool[i][1]) ) # cruzamiento 1 punto
            elif crossover == "uniform":
                offspring_population.extend( mating_pool[i][0].crossover_uniform(mating_pool[i][1]) ) # cruzamiento uniforme
            else:
                raise NotImplementedError

        ## Aplica el operador de mutacion con probabilidad pmut en cada hijo generado
        for i in range(len(offspring_population)):
            if uniform(0, 1) < pmut: 
                if mutation == "flip":
                    offspring_population[i] = offspring_population[i].mutation_flip() # cambia el alelo de un gen
                elif mutation == "multiflip":
                    offspring_population[i] = offspring_population[i].mutation_multiflip() # cambia el alelo de varios genes
                elif mutation == "inversion":
                    offspring_population[i] = offspring_population[i].mutation_inversion() # invierte todos los genes entre 2 puntos al azar
                else:
                    raise NotImplementedError   
        
        ## Evalua la poblacion descendencia creada
        evaluate_population_mono(offspring_population)   # evalua la poblacion descendencia
        
        ## Selecciona individuos para la sgte. generación 
        if selection_survivors_method == "ranking":
            population = select_survivors_ranking(population, offspring_population, popsize) #metodo de ranking
        else:
            raise NotImplementedError
            
        ## Almacena la historia del fitness del mejor individuo
        ibest = sorted(range(len(population)), key=lambda i: population[i].fitness, reverse=True)[:1]
        bestfitness.append(population[ibest[0]].fitness)
        
        if (g % 10 == 0):  # muestra resultados cada 10 generaciones
            print("generacion {}, (Mejor fitness = {})".format(g, population[ibest[0]].fitness))
        
    print("Mejor individuo en la ultima generacion = {} (fitness = {})".format(population[ibest[0]].chromosome, population[ibest[0]].fitness))
    return population[ibest[0]], bestfitness  # devuelve el mejor individuo y la lista de mejores fitness x gen

## Solución


### Ejecución del algoritmo Multi Objetivo


In [25]:
## Hiperparametros del algoritmo genetico
from copy import deepcopy

#POP_SIZE = 50
MIN_POP_SIZE = 100
MAX_POP_SIZE = 100
GENERATIONS = 300   # numero de generaciones
PMUT = 0.3         # tasa de mutacion

P = init_population( MAX_POP_SIZE )   # Crea  una poblacion inicial
population=deepcopy(P)
#  evalua la poblacion inicial
evaluate_population_multi(P)

# Obtiene la poblacion de la frontera de pareto final 
pareto_front_population=genetic_algorithm_multi(P, GENERATIONS, PMUT,"uniform", "flip",MIN_POP_SIZE, MAX_POP_SIZE)

Generacion 0 (de 300) 
Generacion 10 (de 300) 
Generacion 20 (de 300) 
Generacion 30 (de 300) 
Generacion 40 (de 300) 
Generacion 50 (de 300) 
Generacion 60 (de 300) 
Generacion 70 (de 300) 
Generacion 80 (de 300) 
Generacion 90 (de 300) 
Generacion 100 (de 300) 
Generacion 110 (de 300) 
Generacion 120 (de 300) 
Generacion 130 (de 300) 
Generacion 140 (de 300) 
Generacion 150 (de 300) 
Generacion 160 (de 300) 
Generacion 170 (de 300) 
Generacion 180 (de 300) 
Generacion 190 (de 300) 
Generacion 200 (de 300) 
Generacion 210 (de 300) 
Generacion 220 (de 300) 
Generacion 230 (de 300) 
Generacion 240 (de 300) 
Generacion 250 (de 300) 
Generacion 260 (de 300) 
Generacion 270 (de 300) 
Generacion 280 (de 300) 
Generacion 290 (de 300) 


### Ploteo de los individuos en la frontera de Pareto

In [ ]:
## Plotea los individuos de la frontera de Pareto final
pop_size = len(pareto_front_population)
num_objectives = len(pareto_front_population[0].fitness)
    
# extrae los fitness de la poblacion en la matriz fitnesses
fitnesses = np.zeros([pop_size, num_objectives])
for i in range(pop_size): fitnesses[i,:] = pareto_front_population[i].fitness

x = fitnesses[:, 0]
y = fitnesses[:, 1]
plt.xlabel('Objectivo A - 1(Horas libre Profesores) ')
plt.ylabel('Objectivo B - 1/(Horas libre Grupo) ')
plt.scatter(x,y)
#plt.savefig('pareto.png')
plt.show()

In [ ]:
#Algunos de los valores del pareto front y su fitness:
# for i in range(5):
#     b = pareto_front_population[i].fitness
#     print(b)

In [ ]:
#Algunas de de las soluciones posibles:
# for i in range(5):
#     a = pareto_front_population[i].chromosome
#     print(a)

### Ejecución del algoritmo Mono Objetivo

Para esto se uso la misma población inicial que el algoritmo anterior

In [ ]:
 ## Hiperparametros del algoritmo genetico
POPSIZE = 100       # numero de individuos
GENERATIONS = 300   # numero de generaciones
PMUT = 0.3       # taza de mutacion


## Inicializa una poblacion inicial de forma aleatoria
# Evolue la poblacion con el algoritmo genetico (cruzamiento 'onepoint', )
population = init_population( POPSIZE)   # Crea  una poblacion inicial
best_ind, bestfitness = genetic_algorithm_mono(population, GENERATIONS, PMUT, 
                                          crossover="onepoint", mutation="multiflip", 
                                          selection_parents_method = 'tournament', 
                                          selection_survivors_method = 'ranking')

print(best_ind, bestfitness)
# muestra la evolucion del mejor fitness
plt.plot(bestfitness)
plt.show()
solucion=best_ind.chromosome
print(solucion)

### Se halla el Fitness Multiobjetivo para el mejor individuo encontrado por al algoritmo Mono Objetivo (para poder hacer el ploteo)

In [ ]:
#print(best_ind.chromosome)
#print(best_ind.fitness)
fitness_mono=get_fitness_multi(best_ind.chromosome)



### Ploteo del mejor individuo del Mono Objetivo junto con los resultados del Multi Objetivo

In [ ]:
## Plotea los individuos de la frontera de Pareto final
pop_size = len(pareto_front_population)
num_objectives = len(pareto_front_population[0].fitness)
    
# extrae los fitness de la poblacion en la matriz fitnesses
fitnesses = np.zeros([pop_size, num_objectives])
for i in range(pop_size): fitnesses[i,:] = pareto_front_population[i].fitness

x = fitnesses[:, 0]
y = fitnesses[:, 1]
plt.xlabel('Objectivo A - 1(Horas libre Profesores) ')
plt.ylabel('Objectivo B - 1/(Horas libre Grupo) ')
plt.scatter(x,y)
#plt.savefig('pareto.png')

plt.plot([fitness_mono[0]], [fitness_mono[1]], 'ro')
plt.show()

plt.show()

### Visualización del Horario generado por el algoritmo Mono Objetivo

In [ ]:
#imprimir_horario(solucion)

### Visualización de un Horario generado por el algoritmo Multi Objetivo

In [ ]:
#Algunas de de las soluciones posibles:
i=choice(pareto_front_population)
#solucion_multi = i.chromosome
#imprimir_horario(solucion_multi)

In [ ]:
conflictos, horas_libres_profesores, horas_libres_grupos, horas_premium, dias_ocupados_profesores, dias_ocupados_grupos, dias_no_usados, profesor_extras= Genes.evaluacion_cromosoma(i.chromosome)
print(f"Resumen de cromosoma\nConflictos={conflictos}\nHuecos para profesores={horas_libres_profesores}\nHuevos para grupos={horas_libres_grupos}\nHoras premium usadas={horas_premium}\nDias ocupados por profesores = {dias_ocupados_profesores}\nDias ocupados por alumnos = {dias_ocupados_grupos}\n N° dias no usados = {dias_no_usados}\nN° horas extra usadas = {profesor_extras}")

In [ ]:
## Clase para prueba Iterativa 
import itertools
from itertools import chain, combinations

class IterativeSearch(object):

  def __init__(self, **kwargs):

    # dict con los params en común entre multi y mono 
    variableBaseCommonEvolutiveParams = {
        "min_pop_size_list": [100,300],
        "max_pop_size_list": [100,300],
        "ngen_list": [300, 500, 1000], 
        "pmut_list": [0.1, 0.5, 0.8],
        "crossover_list": ["uniform", "onepoint"],
        "mutation_list": ["flip", "multiflip"],
        "selection_parents_method_list": ["tournament"] #"roulette" presenta problemas
    }

    # ingesta de parámetros
    for (prop, default) in variableBaseCommonEvolutiveParams.items():
      setattr(self, prop, kwargs.get(prop, default))     

    # ingestando parámetro de suvivorship único en formato lista
    self.selection_survivors_method_list = ['ranking']

  # obtiene todas las posibles comb. según parámetros definidos
  def __allCombinations__(self):

    # list containing all elements
    base_list = [
                   self.min_pop_size_list, #ord 0
                   self.max_pop_size_list, #ord 1
                   self.ngen_list, #ord 2
                   self.pmut_list, #ord 3
                   self.crossover_list, #ord 4
                   self.mutation_list, #ord 5
                   self.selection_parents_method_list, #ord 6
                   self.selection_survivors_method_list #ord 7
                   ]
    return list(itertools.product(*base_list))  

  # método de iteración para gen. multiobjetivo
  def getMultiObjective(self):
    results = []
    combinationParamsResults = []
    for idx, paramsCombination in enumerate(self.__allCombinations__()):
      print(f"------------>> TRIAL {idx} <<------------")
      print("Combinación de parámetros:")
      print(" ---> ", paramsCombination)

      # usa máx pop size como inicializador de población
      P = init_population( paramsCombination[1] ) 

      # Population DeepCopy
      population=deepcopy(P)

      #  evalua la poblacion inicial
      evaluate_population_multi(P)      

      # ejecución del algoritmo multiobjetivo
      pareto_front_population = genetic_algorithm_multi(
          population = P, 
          ngen= paramsCombination[2], 
          pmut= paramsCombination[3],
          crossover= paramsCombination[4], 
          mutation= paramsCombination[5], 
          min_pop_size= paramsCombination[0], 
          max_pop_size=paramsCombination[1]
          )
      
      # guardando resultados
      results.append(pareto_front_population)
      combinationParamsResults.append(paramsCombination)

    return results, combinationParamsResults

  # método de iteración para gen. multiobjetivo
  def getMonoObjective(self):
    results_ind, results_fit = [], []
    combinationParamsResults = []
    for idx, paramsCombination in enumerate(self.__allCombinations__()):
      print(f"------------>> TRIAL {idx} <<------------")
      print("Combinación de parámetros:")
      print(" ---> ", paramsCombination)      
      
      # usa máx pop size como inicializador de población
      P = init_population( paramsCombination[1] ) 
      
      # ejecutando algoritmo monoobjetivo
      best_ind, bestfitness = genetic_algorithm_mono(
          population= P, 
          ngen= paramsCombination[2], 
          pmut= paramsCombination[3], 
          crossover= paramsCombination[4], 
          mutation= paramsCombination[5], 
          selection_parents_method = paramsCombination[6], 
          selection_survivors_method = paramsCombination[7]
          )
    
      # guardando resultados de mejor individuo
      results_ind.append(best_ind)

      # guardando resultados de mejor fitness      
      results_fit.append(bestfitness)

      # guardando combinaciones de params
      combinationParamsResults.append(paramsCombination)      
    return results_ind, results_fit, combinationParamsResults

### Probando solución Iterativa | Mono-objetivo

In [ ]:
# ejemplo con Mono objetivo
bestinds, bestfits, bestParamsMono = IterativeSearch().getMonoObjective()

for best_ind, bestfitness in zip(bestinds, bestfits):
  print(f"Best Fitness {bestfitness[-1]}")
  plt.plot(bestfitness)
  plt.show()
  solucion=best_ind.chromosome
  print(solucion)

# best individual - Mono Objetivo
bestFitnessByComb = list(map(lambda x: x[-1], bestfits))
idxBestFitMono = bestFitnessByComb.index(max(bestFitnessByComb))
print(f"Best Fitness Mono: {bestFitnessByComb[idxBestFitMono]}")
print(f"Best Params Combination: {bestParamsMono[idxBestFitMono]}")
print(f"Best Individual Mono:{bestinds[idxBestFitMono].chromosome}")

### Probando solución Iterativa | Multi-objetivo

In [ ]:
bestParetoFrontier, bestParamsMulti = IterativeSearch().getMultiObjective()

### Exportar Horario en CSV


In [ ]:
horario = exportar_horario(solucion)
horario.to_csv("horario_n1.csv", header=None, index=False)